In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import time
import copy


In [ ]:
# Define transformations and load datasets
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
     ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Specify the data directory
data_path = "data"

# Load the datasets with the corresponding transformations
flowers_train_data = datasets.Flowers102(root=data_path, split='train', download=True, transform=data_transforms['train'])
flowers_val_data = datasets.Flowers102(root=data_path, split='val', download=True, transform=data_transforms['valid'])
flowers_test_data = datasets.Flowers102(root=data_path, split='test', download=True, transform=data_transforms['test'])

# Create data loaders for each split with specific batch size and shuffling settings
dataloaders = {
    'train': DataLoader(flowers_train_data, batch_size=128, shuffle=True),
    'val': DataLoader(flowers_val_data, batch_size=128, shuffle=False),
    'test': DataLoader(flowers_test_data, batch_size=128, shuffle=False)
}

dataset_sizes = {
    'train': len(flowers_train_data),
    'val': len(flowers_val_data),
    'test': len(flowers_test_data)
}



100%|██████████| 344862509/344862509 [00:10<00:00, 32311859.65it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 816417.45it/s]


100%|██████████| 14989/14989 [00:00<00:00, 21369280.30it/s]


In [ ]:
# Define the CNN model
class FlowerCNN(nn.Module):
    def __init__(self):
        super(FlowerCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.conv5 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(512 * 7 * 7, 4096)
        self.bn6 = nn.BatchNorm1d(4096)
        self.fc2 = nn.Linear(4096, 102)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        x = self.pool(torch.relu(self.bn4(self.conv4(x))))
        x = self.pool(torch.relu(self.bn5(self.conv5(x))))
        x = x.view(-1, 512 * 7 * 7)
        x = self.dropout(torch.relu(self.bn6(self.fc1(x))))
        x = self.fc2(x)
        return x

In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Initialize the model
model = FlowerCNN().to(device)


In [ ]:

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

In [ ]:



# Decay learning rate by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=7)

def train_model(model, criterion, optimizer, scheduler, num_epochs=150):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                # Track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} loss: {:.4f} accuracy: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training completed in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val accuracy: {:4f}'.format(best_acc))

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Train the model
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=150)

Epoch 0/149
----------
train loss: 4.3614 accuracy: 0.0529
val loss: 4.6725 accuracy: 0.0098

Epoch 1/149
----------
train loss: 3.5426 accuracy: 0.1716
val loss: 4.7031 accuracy: 0.0245

Epoch 2/149
----------
train loss: 3.2023 accuracy: 0.2255
val loss: 4.3287 accuracy: 0.0412

Epoch 3/149
----------
train loss: 2.9579 accuracy: 0.2814
val loss: 3.6928 accuracy: 0.1176

Epoch 4/149
----------
train loss: 2.7434 accuracy: 0.3480
val loss: 3.1555 accuracy: 0.2304

Epoch 5/149
----------
train loss: 2.6706 accuracy: 0.3706
val loss: 2.7945 accuracy: 0.3441

Epoch 6/149
----------
train loss: 2.5587 accuracy: 0.3853
val loss: 2.6691 accuracy: 0.3735

Epoch 7/149
----------
train loss: 2.5447 accuracy: 0.3941
val loss: 2.6498 accuracy: 0.3833

Epoch 8/149
----------
train loss: 2.5709 accuracy: 0.3892
val loss: 2.6301 accuracy: 0.3833

Epoch 9/149
----------
train loss: 2.5423 accuracy: 0.3833
val loss: 2.5975 accuracy: 0.3873

Epoch 10/149
----------
train loss: 2.4248 accuracy: 0.4324


In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'flowers102_classifier_model.pth')



In [ ]:
# Load the trained model for evaluation
model.load_state_dict(torch.load('flowers102_classifier_model.pth'))
model.eval()



FlowerCNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_fe

In [ ]:
# Function to evaluate accuracy
def evaluate_model(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient calculation for evaluation
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy



In [ ]:
# Evaluate the model on the test set
test_accuracy = evaluate_model(model, dataloaders['test'])
print(f'Test Accuracy: {test_accuracy:.2f}%')

Test Accuracy: 55.34%
